In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter
from itertools import cycle
import os
import time
import math
import pandas as pd
import wandb
from collections import OrderedDict
from sklearn.linear_model import LogisticRegression
    
import copy
import torch.nn as nn
from torch.autograd import Variable
from typing import List
import itertools
from tqdm.autonotebook import tqdm
#from models import *
from unlearning.unlearning_benchmarks.SCRUB import models, utils
#import models

#from logger import *
#import utils


from unlearning.unlearning_benchmarks.SCRUB.thirdparty.repdistiller.helper.util import adjust_learning_rate as sgda_adjust_learning_rate
from unlearning.unlearning_benchmarks.SCRUB.thirdparty.repdistiller.distiller_zoo import DistillKL, HintLoss, Attention, Similarity, Correlation, VIDLoss, RKDLoss
from unlearning.unlearning_benchmarks.SCRUB.thirdparty.repdistiller.distiller_zoo import PKT, ABLoss, FactorTransfer, KDSVD, FSP, NSTLoss

from unlearning.unlearning_benchmarks.SCRUB.thirdparty.repdistiller.helper.loops import train_distill, train_distill_hide, train_distill_linear, train_vanilla, train_negrad, train_bcu, train_bcu_distill, validate

#from unlearning.unlearning_benchmarks.SCRUB.thirdparty.repdistiller.helper.pretrain import init
import numpy as np
import torch

In [ ]:
import os
import shutil
import yaml
import numpy as np
import torch as ch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import logging
import pprint
from contextlib import redirect_stdout, redirect_stderr

from unlearning.auditors.utils import (
    model_factory,
    loader_factory,
    load_forget_set_indices,
    get_full_model_paths,
    get_oracle_paths,
    make_results_dir,
)
from unlearning.auditors.accuracies import eval_accuracy
from unlearning.auditors.logit_plots import compute_logits, plot_logits
from unlearning.auditors.basic import plot_margins
from unlearning.auditors.ulira import cheap_ulira_audit_precomputed
from unlearning.auditors.direct import (
    config_submitit,
    direct_audit_precomputed,
    get_u_margins,
    u_margin_job,
    plot_margins_direct,
)
from unlearning.unlearning_algos.base_nn import NAME_TO_ALGO




def read_yaml(yaml_file):
    with open(yaml_file, "r") as f:
        config = yaml.safe_load(f)
    return config

def load_model(path, model_factory, ds_name):
    model = model_factory(ds_name)
    loaded_model = ch.load(path)
    first_key = list(loaded_model.keys())[0]
    if "model" in first_key:
        model.load_state_dict(loaded_model)

    else:
        # add ".model" to each key in k,vs
        loaded_model = {f"model.{k}": v for k, v in loaded_model.items()}
        model.load_state_dict(loaded_model)
    return model


def get_model_and_loaders(forget_set_id= 4, ds_name = "CIFAR10"):

    # for now, let's tie the model to the dataset, so we have fewer moving pieces
    model = model_factory(ds_name)  # on cuda, in eval mode
    
    forget_set_indices = load_forget_set_indices(ds_name,forget_set_id)
    print(f"getting the dataloaders")
    with redirect_stdout(open("/dev/null", "w")):
        # no shuffling, no augmentation
        train_loader = loader_factory(ds_name, indexed=False)
        val_loader = loader_factory(ds_name, split="val", indexed=False)
        total_train_set = len(train_loader.dataset)
        retain_inds = np.setdiff1d(np.arange(total_train_set), forget_set_indices)
        retain_loader = loader_factory(ds_name, indices=retain_inds, indexed=False)

        forget_loader = loader_factory(
            ds_name,
            indices=forget_set_indices,
            batch_size=50,
            indexed=False,
        )
        eval_set_inds = np.arange(
            len(train_loader.dataset) + len(val_loader.dataset))
        eval_loader = loader_factory(ds_name,
                                     split="train_and_val",
                                     indices=eval_set_inds,
                                     indexed=False)
    # inserted by Roy for some speed reason
    splits = ["train", "val"]
    print(f"loading the model!")
    f_ckpt_paths, f_logit_paths, f_margins_paths = get_full_model_paths(
        ds_name, splits=splits)
    (
        o_ckpt_0_path,  # we only need a single oracle checkpoint
        o_logit_paths,
        o_margins_paths,
    ) = get_oracle_paths(ds_name, forget_set_id, splits=splits)
    print(f"Loaded paths of pretrained models.")

    full_model = load_model(f_ckpt_paths[0], model_factory, ds_name)
    return full_model, (train_loader, val_loader, forget_loader, retain_loader, eval_loader), forget_set_indices
        
forget_set_id = 4

full_model, loaders, forget_set_indices = get_model_and_loaders(forget_set_id= forget_set_id,  ds_name = "CIFAR10")
(train_loader, val_loader, forget_loader, retain_loader, eval_loader) = loaders 



In [ ]:
from argparse import Namespace
args = Namespace()


In [ ]:
args.optim = 'adam'
args.gamma = 1
args.alpha = 0.5
args.beta = 0
args.smoothing = 0.5
args.msteps = 3
args.clip = 0.2
args.sstart = 10
args.kd_T = 2
args.distill = 'kd'

args.sgda_epochs = 10
args.sgda_learning_rate = 0.0005
args.lr_decay_epochs = [5,8,9]
args.lr_decay_rate = 0.1
args.sgda_weight_decay = 0.1#5e-4
args.sgda_momentum = 0.9

# TODO: teacher, student
# teacher = None, student = None 
# TODO : datasets and dataloaders 

teacher = copy.deepcopy(full_model)
student = copy.deepcopy(full_model)

# load up model 1 
model_t = copy.deepcopy(teacher)
model_s = copy.deepcopy(student)

module_list = nn.ModuleList([])
module_list.append(model_s)
trainable_list = nn.ModuleList([])
trainable_list.append(model_s)

criterion_cls = nn.CrossEntropyLoss()
criterion_div = DistillKL(args.kd_T)
criterion_kd = DistillKL(args.kd_T)


criterion_list = nn.ModuleList([])
criterion_list.append(criterion_cls)    # classification loss
criterion_list.append(criterion_div)    # KL divergence loss, original knowledge distillation
criterion_list.append(criterion_kd)     # other knowledge distillation loss


optimizer = torch.optim.Adam(trainable_list.parameters(),
                        lr=args.sgda_learning_rate,
                        weight_decay=args.sgda_weight_decay)



acc_rs = []
acc_fs = []
acc_ts = []


def _adjust_learning_rate(epoch, optimizer):
    """Sets the learning rate to the initial LR decayed by decay rate every steep step"""
    steps = np.sum(epoch > np.asarray(args.lr_decay_epochs))
    new_lr = args.sgda_learning_rate
    if steps > 0:
        new_lr = args.sgda_learning_rate * (args.lr_decay_rate ** steps)
        for param_group in optimizer.param_groups:
            param_group['lr'] = new_lr
    return new_lr

beta = 0.1
def avg_fn(averaged_model_parameter, model_parameter, num_averaged): return (
    1 - beta) * averaged_model_parameter + beta * model_parameter
swa_model = torch.optim.swa_utils.AveragedModel(
    model_s, avg_fn=avg_fn)




In [ ]:
print(f"total epochs : {args.sgda_epochs}")
for epoch in range(1, args.sgda_epochs + 1):
    print(f"Epoch {epoch} ...")
    lr = _adjust_learning_rate(epoch, optimizer)

    print("==> scrub unlearning ...")

    print(f"validating - ")
    acc_r, acc5_r, loss_r = validate(retain_loader, model_s, criterion_cls, args, True)
    acc_f, acc5_f, loss_f = validate(forget_loader, model_s, criterion_cls, args, True)
    acc_rs.append(100-acc_r.item())
    acc_fs.append(100-acc_f.item())

    maximize_loss = 0
    print(f"train distill 1")ß
    if epoch <= args.msteps:
        maximize_loss = train_distill(epoch, forget_loader, module_list, swa_model, criterion_list, optimizer, args, "maximize")
    print(f"train distill 2 :")
    train_acc, train_loss = train_distill(epoch, retain_loader, module_list, swa_model, criterion_list, optimizer, args, "minimize",)
    if epoch >= args.sstart:
        print("update params")
        swa_model.update_parameters(model_s)

    print("maximize loss: {:.2f}\t minimize loss: {:.2f}\t train_acc: {}".format(maximize_loss, train_loss, train_acc))